In [1]:
import pandas as pd

### Importando Datasets

In [2]:
df_movies = pd.read_csv("movies.csv", sep=",")
df_ratings = pd.read_csv("ratings.csv", sep=",")

### Analisando o dataframe Movies

In [3]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


### Analisando o dataframe Ratings

In [5]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


### Quantos gêneros o filme "The Matrix" tem? Quais são os gêneros?

In [7]:
# Procurando no dataframe pelos títulos que contêm a palavra Matrix
mask_movie_matrix = df_movies['title'].str.contains('Matrix')
df_movies[mask_movie_matrix]

,movieId,title,genres
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
4351,6365,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX
4639,6934,"Matrix Revolutions, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX


In [8]:
# Foi encontrado que o Id do filme Matrix é 2571
df_matrix = df_movies.query('movieId == 2571')

# Separando os gêneros do filme Matrix em uma lista
genres = df_matrix['genres'].str.split('|').to_list()[0]

In [9]:
# Respostas
print(f"O filme Matrix tem {len(genres)} gêneros, os quais são: {', '.join(genres)}")

O filme Matrix tem 3 gêneros, os quais são: Action, Sci-Fi, Thriller


### Quantos gêneros cada filme tem, em média?

In [10]:
def find_number_of_genres(dataframe: pd.DataFrame) -> list:
    """Encontra o número de gêneros para cada filme do dataframe
    Args:
        dataframe (pd.DataFrame): Dataframe a ser utilizado

    Returns:
        list: Lista contendo o número de gêneros para cada filme
    """    
    movies_genres = dataframe['genres'].str.split('|').to_list()
    number_of_genres = [len(genres) for genres in movies_genres]
    
    return number_of_genres

In [11]:
# Adiciona a lista com o número de gêneros de cada filme ao dataframe
df_movies['n_genres'] = find_number_of_genres(df_movies)

In [12]:
# Resposta
print(f"Cada filme tem em média {df_movies['n_genres'].mean():.2f} gêneros")

Cada filme tem em média 2.27 gêneros


### Quais são os cinco filmes com maior número de gêneros?

In [13]:
# Calculando os 5 filmes com maior número de gêneros
top_genres_movies = df_movies[['title','genres','n_genres']].nlargest(5,'n_genres')
top_genres_movies

,title,genres,n_genres
7441,Rubber (2010),Action|Adventure|Comedy|Crime|Drama|Film-Noir|...,10
5556,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Action|Animation|Crime|Drama|Film-Noir|Mystery...,8
1390,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...,7
2250,Who Framed Roger Rabbit? (1988),Adventure|Animation|Children|Comedy|Crime|Fant...,7
3460,Osmosis Jones (2001),Action|Animation|Comedy|Crime|Drama|Romance|Th...,7


In [14]:
# Convertendo para lista
answer = top_genres_movies['title'].to_list()
print(f"Os filmes com maior número de gêneros são: {', '.join(answer)}")

Os filmes com maior número de gêneros são: Rubber (2010), Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989), Mulan (1998), Who Framed Roger Rabbit? (1988), Osmosis Jones (2001)


### Quantos usuários deram nota para o filme "The Matrix"?

In [15]:
id_matrix = 2571
# Pesquisando em ratings pelo id em df_ratings
reviews_matrix = df_ratings.query('movieId == @id_matrix')

# Resposta
print(f"{len(reviews_matrix)} usuários avaliaram o filme The Matrix.")

278 usuários avaliaram o filme The Matrix.


### Quantos usuários, em média, deram nota para cada filme?

In [18]:
average_users_review = len(df_ratings)/len(df_movies)

In [19]:
# Resposta
print(f"Em média, {average_users_review:.2f} usuários deram nota para cada filme.")

Em média, 10.35 usuários deram nota para cada filme.


### Quais são os cinco filmes para os quais mais usuários deram nota?

In [20]:
# Calcula os 5 filmes mais avaliados
rated_movies = df_ratings.groupby(['movieId']).agg({'rating':'count'})
top_5_rated = rated_movies.nlargest(5,'rating').rename(columns={'rating':'n_reviews'})
top_5_rated

,n_reviews
movieId,
356,329
318,317
296,307
593,279
2571,278


In [21]:
# Faz um join com o dataframe de movies para pegar o título
top_movies= top_5_rated.merge(df_movies, how='left', on='movieId')
top_movies = top_movies[['n_reviews','title']]
top_movies

,n_reviews,title
0,329,Forrest Gump (1994)
1,317,"Shawshank Redemption, The (1994)"
2,307,Pulp Fiction (1994)
3,279,"Silence of the Lambs, The (1991)"
4,278,"Matrix, The (1999)"


In [22]:
# Convertendo para lista
answer = top_movies['title'].to_list()
print(f"Os filmes com maior número de avaliações são: {', '.join(answer)}")

Os filmes com maior número de avaliações são: Forrest Gump (1994), Shawshank Redemption, The (1994), Pulp Fiction (1994), Silence of the Lambs, The (1991), Matrix, The (1999)


### Qual o rating médio do filme "The Matrix"?

In [23]:
id_matrix = 2571
review_matrix = df_ratings.query('movieId == @id_matrix') 
print(f"O rating médio do filme The Matrix é {review_matrix['rating'].mean():.2f}")

O rating médio do filme The Matrix é 4.19


### Cada filme tem um rating médio. Qual a média desses ratings, por gênero?

In [25]:
def find_genres(dataframe: pd.DataFrame) -> list:
    """Encontra os gêneros de cada filme do dataframe
    Args:
        dataframe (pd.DataFrame): Dataframe a ser utilizado

    Returns:
        list: Lista contendo os gêneros para cada filme
    """    
    movies_genres = dataframe['genres'].str.split('|').to_list()
    return movies_genres

In [26]:
df_movies_genres = df_movies.copy()
df_movies_genres['genres'] = find_genres(df_movies)
df_movies_genres = df_movies_genres.explode('genres')
df_movies_genres

,movieId,title,genres,n_genres
0,1,Toy Story (1995),Adventure,5
0,1,Toy Story (1995),Animation,5
0,1,Toy Story (1995),Children,5
0,1,Toy Story (1995),Comedy,5
0,1,Toy Story (1995),Fantasy,5
...,...,...,...,...
9738,193583,No Game No Life: Zero (2017),Fantasy,3
9739,193585,Flint (2017),Drama,1
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action,2
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Animation,2


In [27]:
df_all_info = df_movies_genres.merge(df_ratings, how='left', on='movieId')
df_genre_avg_rating = df_all_info.groupby(['genres']).agg({'rating':'mean'}).rename(columns={'rating':'avg_rating'})
df_genre_avg_rating

,avg_rating
genres,
(no genres listed),3.489362
Action,3.447984
Adventure,3.508609
Animation,3.629937
Children,3.412956
Comedy,3.384721
Crime,3.658294
Documentary,3.797785
Drama,3.656184


### Para cada gênero de filme, quantos usuários deram cada nota possível?

In [28]:
df_genre_rating = df_all_info.groupby(['genres','rating']).agg({'rating':'count'}).rename(columns={'rating':'n_users'})
df_genre_rating

n_users
genres             rating         
(no genres listed) 0.5           2
                   1.0           2
                   2.0           2
                   2.5           6
                   3.0           6
...                            ...
Western            3.0         442
                   3.5         201
                   4.0         543
                   4.5         158
                   5.0         295

[199 rows x 1 columns]